In [ ]:
import os
import sys
import dlib
import cv2
import math
from tqdm.auto import tqdm

# location of the model (path of the model).
model_path = 'shape_predictor_68_face_landmarks.dat'

# catalogues (global values)
face_image_catalogue = './samples'
occ_catalogue = './occ'
result_catalogue = './results'

# filename ending for images with synthetic occlusion applied
synthetic_file_ext = ['_glass.', '_mask.','_cap.', '_sunglass.']

# occlusions choices (filename.ext in the occ catalog, like 'glass.png')
mask = 'mask.png'
cap = ''
glass = 'glass.png'

In [ ]:
# Prining/log functions
def print_message(error_message):
    ''' Print all error messages from the program run '''
    if not error_message:
        print('None errors during occlusion masking')
    else:
        print('Number of files skipped (error):', len(error_message))
        print('Errors detected during program run:', file=sys.stderr)
        print(*error_message, sep="\n", file=sys.stderr)
    return None


def print_result(error_message, generated_images_count):
    ''' Print overview of program run '''
    print('Occlusion(s) added: ', end='')
    if mask:
        print('Mask ', end='')
    if cap:
        print('Cap ', end='')
    if glass:
        print('Glass', end='')
    print('\nNumber of synthetic images generated:', generated_images_count)

    print_message(error_message)
    return None

In [ ]:
def maximum_area_of_rotated(width, height, angle):
    '''
    Given a rectangle of size wxh that has been rotated by 'angle' (in
    radians), computes the width and height of the largest possible
    axis-aligned rectangle (maximal area) within the rotated rectangle.
    Original code by coproc from Stack Overflow, but adapted for this project
    '''
    if width <= 0 or height <= 0:
      return 0,0

    width_is_longer = (width >= height)
    side_long, side_short = (width,height) if width_is_longer else (height,width)

    # since the solutions for angle, -angle and 180-angle are all the same,
    # if suffices to look at the first quadrant and the absolute values of sin,cos:
    sin_a, cos_a = abs(math.sin(angle)), abs(math.cos(angle))
    if side_short <= 2.*sin_a*cos_a*side_long or abs(sin_a-cos_a) < 1e-10:
      # half constrained case: two crop corners touch the longer side,
      #   the other two corners are on the mid-line parallel to the longer line
      x = 0.5*side_short
      return (x/sin_a,x/cos_a) if width_is_longer else (x/cos_a,x/sin_a)
    else:
      # fully constrained case: crop touches all 4 sides
      cos_2a = cos_a*cos_a - sin_a*sin_a
      return (width*cos_a - height*sin_a)/cos_2a, (height*cos_a - width*sin_a)/cos_2a

def rotate_img(img, angle):
    ''' Rotate given image by angle angle in range 0...360 degree.
    Returns: rotated image '''
    # dividing height and width by 2 to get the center of the image
    height, width = img.shape[:2]
    # get the center coordinates of the image to create the 2D rotation matrix
    center = (width/2, height/2)

    # using cv2.getRotationMatrix2D() to get the rotation matrix
    rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=angle, scale=1)

    return cv2.warpAffine(src=img, M=rotate_matrix, dsize=(width, height))


def rotate_and_crop(img, angle, crop):
    ''' rotate image
    Parameters:
        img: cv2 image matrix object
        angle: in degree
        crop: True cropping of the rotated image
              False rotate and then crop image
    '''
    if not crop:
        return rotate_img(img, angle)   # rotate only
    else:
        optimal_width, optimal_height = maximum_area_of_rotated(img.shape[1], img.shape[0],
                                    math.radians(angle))
        rotated = rotate_img(img, angle)
        height, width, = rotated.shape[:2]
        y1 = height//2 - int(optimal_height/2)
        y2 = y1 + int(optimal_height)
        x1 = width//2 - int(optimal_width/2)
        x2 = x1 + int(optimal_width)

        return rotated[y1:y2, x1:x2]

def calculate_eye_center(detected_landmarks):
    ''' Calculate center of each eye based on all 6 eye FaceLandmarks
    returns:
        lefteye_center (y,x), right_eye_center (y,x)
    '''
    # Left eye centre
    left_eye_x = int((detected_landmarks.part(36).x + detected_landmarks.part(37).x +
                      detected_landmarks.part(38).x + detected_landmarks.part(39).x +
                      detected_landmarks.part(40).x + detected_landmarks.part(41).x)/ 6)
    left_eye_y = int((detected_landmarks.part(36).y + detected_landmarks.part(37).y +
                      detected_landmarks.part(38).y + detected_landmarks.part(39).y +
                      detected_landmarks.part(40).y + detected_landmarks.part(41).y ) / 6)
    # Right Eye centre
    right_eye_x = int((detected_landmarks.part(42).x + detected_landmarks.part(43).x +
                       detected_landmarks.part(44).x + detected_landmarks.part(45).x +
                       detected_landmarks.part(46).x + detected_landmarks.part(47).x) / 6)
    right_eye_y = int((detected_landmarks.part(42).y + detected_landmarks.part(43).y +
                       detected_landmarks.part(44).y + detected_landmarks.part(45).y +
                       detected_landmarks.part(46).y + detected_landmarks.part(47).y) / 6)

    left_eye_center = (left_eye_y, left_eye_x)
    right_eye_center = (right_eye_y, right_eye_x)

    return (left_eye_center, right_eye_center)

def rotate_face_image_angle(face_img, detected_landmarks):
    ''' Calculate rotate angel of face image
    return: angle: in range 0..360
    '''

    def get_angel (a, b, c):
        ''' Calculate angel between 3 pojnts (2-dimensional '''
        ang = math.degrees(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
        return ang + 360 if ang < 0 else ang

    left_eye_center, right_eye_center = calculate_eye_center(
        detected_landmarks)

    right_eye_fixed = (right_eye_center[0], left_eye_center[1])

    return get_angel(left_eye_center, right_eye_center, right_eye_fixed)


In [ ]:
def add_mask_on_image(bg_img, mask_img, alpha_img, alpha_mask_img, x, y):
    ''' Adding a mask onto face image
    Parameters:
        bg_img : Face image
        mask_img : occlusion image
        alpha_img : alpha notation of the mask
        alpha_mask_img : mask of the image (alpha value)
        x : X-coordinat for upper left corner of the occlusion
        y : Y-coordinat for upper left corner of the occlusion
    return:
        image
    '''
    for c in range(0, 3):
        bg_img[y:y+mask_img.shape[0], x:x+mask_img.shape[1], c] = (alpha_mask_img * mask_img[:, :, c] + alpha_img * bg_img[y:y+mask_img.shape[0], x:x+mask_img.shape[1], c])

    return (bg_img)

In [ ]:
def check_if_landmarks_within_image (detected_landmarks, image, image_filename, error_message):
    ''' Check if given landmarks are inside the image
    Parameters
    Returns: True: legal filename
            False: not a legal face image name to be processed
    '''

    # Actual landmarks for testing
    right_ear_x = detected_landmarks.part(0).x
    right_ear_y = detected_landmarks.part(0).y
    left_ear_x = detected_landmarks.part(16).x
    left_ear_y = detected_landmarks.part(16).y
    right_eye_y = detected_landmarks.part(19).y
    left_eye_y = detected_landmarks.part(24).y
    chin_x = detected_landmarks.part(8).x # Only test if mask occlusion is selected
    chin_y = detected_landmarks.part(8).y

    if ((right_ear_x < 0) or (right_ear_y < 0) or (right_ear_x > image.shape[1]) or (right_ear_y > image.shape[0])):
        error_message.append('Image ' + image_filename +

                             ' face Landmarks left ear is out of bounds (i.e. outside the face image)')
        return False, error_message
    if ((left_ear_x < 0) or (left_ear_y < 0) or (left_ear_x > image.shape[1]) or (left_ear_y > image.shape[0])):
        error_message.append('Image ' + image_filename +

                              ' face Landmarks right ear is out of bounds (i.e. outside the face image)')
        return False, error_message
    if (right_eye_y < 0) or (left_eye_y < 0):
        error_message.append('Image ' + image_filename +

                             ' face Landmarks eyes are out of bounds (i.e. outside the face image)')
        return False, error_message
    if mask:
        if ((chin_x < 0) or (chin_y < 0) or (chin_x > image.shape[1]) ):
            error_message.append('Image ' + image_filename +

                                 ' face Landmark chin is out of bounds (i.e. outside the face image)')
            return False, error_message
    return True, error_message


def check_if_face_from_front(detected_landmarks):
    ''' Check if face image photo is taken from front
        calculating the distance from the ear to the center of the eye (both sides)
        if the ratio between the distance values exceeds the ratio, the routine returns False
    return:
        True: the face is from from (within limits) and
        False: the face is from side
    '''

    ratio = 4 # Ratio between ear and eye (keft and right side)
    # get center of eyes:
    left_eye_center , right_eye_center = calculate_eye_center(detected_landmarks)

    right_ear_x = detected_landmarks.part(0).x
    left_ear_x = detected_landmarks.part(16).x
    distance_left = left_eye_center[1] - right_ear_x
    distance_right = left_ear_x - right_eye_center[1]

    if distance_left < distance_right:
        if (distance_right / distance_left) > ratio:
           return False
    if distance_left > distance_right:
        if (distance_left / distance_right) >  ratio:
           return False

    return True


In [ ]:
def mask_glasses(face_img, mask, face_landmarks):
    ''' Mask a glass occlusion on a face image
    parameters:
        img: face image
        mask: occlusion
        face_landmarks: landmarks with 68 points for the face
    return:

        the image with the occlusion
    '''
   

    # positions of the landmarks of interest
    right_ear_x = face_landmarks.part(0).x
    left_ear_x = face_landmarks.part(16).x

   

    # calculate eye centers:
    left_eye_center, right_eye_center = calculate_eye_center(face_landmarks)
   

    '''

    Calculations:
        image_mask_width: with of the mask area in then image
        mask_width: width of the mask (x-direction)

        mask_scale: Scaling of the mask before putting it on the face image
       

        All glasses to be put on image as this:
            X-coordinate left_ear_x-position
            Y-coodinate (middle position of top of eye)
     '''
    # Calculating the sizes and position im face image
    if left_ear_x > face_img.shape[1]:  # right ear (x-position) ouside image!
        left_ear_x = face_img.shape[1]

    image_mask_width = left_ear_x - right_ear_x
    mask_width = mask.shape[1]
   

    mask_scale = (image_mask_width / mask_width)
 

    mask_img = cv2.resize(mask, None, fx=mask_scale, fy=mask_scale)
    y_pos = int((left_eye_center[0] + right_eye_center[0])/2) - int(mask_img.shape[0]/3)
    x_pos = int((right_eye_center[1] + left_eye_center[1])/2)

    x_pos = x_pos - int((mask_img.shape[1]/2))
    if x_pos < 0:  # if calculatet out of bound, set to 0
        mask_img = mask_img[:,abs(x_pos):mask_img.shape[1]] # Crop mask image in x-direction
        x_pos = 0
   

   

    # trying to merge alpha channel
    alpha_mask_img = mask_img[:, :, 3] / 255.0
    alpha_img = 1.0 - alpha_mask_img 

   

    # do the masking
    return add_mask_on_image(face_img, mask_img, alpha_img,alpha_mask_img,x_pos, y_pos)

In [ ]:
def mask_cap (face_img, mask, face_landmarks):
    """ Mask a cap on an face image
    parameters:
        img: face image
        mask: occlusion
        face_landmarks: landmarks with 68 points for the face
    return:
        the image with the occlusion
    """

    # positions of the landmarks of interest
    right_ear_x = face_landmarks.part(0).x
    left_ear_x = face_landmarks.part(16).x
    right_eyebrow_y = face_landmarks.part(19).y
    left_eyebrow_y = face_landmarks.part(24).y
    low_y = face_landmarks.part(27).y # nose root

    # Calculating the sizes and position im face image
    increase_factor = 1.15 # Increase width by 15 % to compensate for ear-position
                             # gives a more realistic outlook

    if left_ear_x > face_img.shape[1]:  # left ear (x-position) ouside image
        left_ear_x = face_img.shape[1]
    if right_ear_x < 0:                 # right ear positioned outside image
        right_ear_x = 0

    # scaling up the occlusion
    image_mask_width = int((left_ear_x - right_ear_x) * increase_factor)
    mask_width = mask.shape[1]
    mask_scale = (image_mask_width / mask_width)

    # Resize the mask for maximum x-size inf face:
    mask_img = cv2.resize(mask, None, fx=mask_scale, fy=mask_scale)
    cap_lower_y_position = int((low_y + (right_eyebrow_y+ left_eyebrow_y)/2)/2)

    """ If the cap (now enlarged) reaches outside the face image, we have to crop the cap
        on both ends (if neccessary) """
    crop_the_occlusion = False # Initially: not a need of cropping the occlusion
    # Need of croppoing the top of the occlusion?
    if cap_lower_y_position < mask_img.shape[0]:
        #Cut off mask on top
        start_pos_y = mask_img.shape[0] - cap_lower_y_position
        crop_the_occlusion = True
        y_pos = 0

    # Need of cropping the occlusion on left and right end?
    x_pos = int(right_ear_x - (image_mask_width*((increase_factor-1)/2)) ) # Left position Adjust by half of increase %
    if (x_pos < 0): # lef position of image is outside the image (must cut left part)
        start_pos_x = -(x_pos) # align the occlusion to the left edge of the face image
        crop_the_occlusion = True
        x_pos = 0
    else:
        start_pos_x = 0

    x_pos_2 = int(left_ear_x + (image_mask_width*((increase_factor-1)/2)) ) # right position Adjust by half of increase %
    if x_pos_2 > face_img.shape[1]:  # Right position of mask is outside the image
        end_pos_x = mask_img.shape[1] -x_pos_2 + face_img.shape[1]
        crop_the_occlusion = True
    else:
        end_pos_x = mask_img.shape[1]

    # Need of cropping the occlusion:
    if crop_the_occlusion:
        # crop the mask image:
        crop_img = mask_img[start_pos_y:mask_img.shape[0],
                       start_pos_x:end_pos_x]
        mask_img = crop_img.copy()

    # Calculate position of the occlusion
    y_pos = cap_lower_y_position - mask_img.shape[0]
    image_mask_width = mask_img.shape[0]
    # Size of the new mask

    alpha_mask_img = mask_img[:, :, 3] / 255.0
    alpha_img = 1.0 - alpha_mask_img

    # occlusion masking:
    return add_mask_on_image(
        face_img, mask_img, alpha_img, alpha_mask_img, x_pos, y_pos)

In [ ]:
def mask_face_mask (face_img, mask, face_landmarks):
    '''

    Mask a face-mask on a face image
    parameters:
        img: face image
        mask: occlusion
        face_landmarks: landmarks with 68 points for the face
    return:

        the image with the occlusion
    '''
      

    # positions of the landmarks of interest
    right_ear_x = face_landmarks.part(0).x
    left_ear_x = face_landmarks.part(16).x
    chin_y  = face_landmarks.part(8).y
    nose_y  = face_landmarks.part(28).y

    # Calculating the sizes and position im face image
    # if left or tight ear is outside the face image, we have to adjust to edge of image
    if left_ear_x > face_img.shape[1]:  # right ear (x-position) ouside image
        left_ear_x = face_img.shape[1]
    if right_ear_x < 0:
        right_ear_x = 0
       

    image_mask_width = left_ear_x - right_ear_x
    mask_width = mask.shape[1]
    image_mask_height = chin_y-nose_y
    mask_height = mask.shape[0]
   

    mask_scale_x = (image_mask_width / mask_width)
    mask_scale_y = (image_mask_height / mask_height)
       

    resized_face_mask = cv2.resize(mask, None, fx=mask_scale_x, fy=mask_scale_y)
    y_pos = chin_y - resized_face_mask.shape[0]
    if chin_y > face_img.shape[0]:
        # have to reduce the mask at bottom, while it exceeds the image size
       resized_face_mask = resized_face_mask[0:(face_img.shape[0] - y_pos),:]

          

    # trying to merge alpha channel
    alpha_mask_img = resized_face_mask[:, :, 3] / 255.0
    alpha_img = 1.0 - alpha_mask_img


    x_pos = right_ear_x # x-position for the mask
   

    # Mask the face:
    return add_mask_on_image(
        face_img, resized_face_mask, alpha_img, alpha_mask_img, x_pos, y_pos)

In [ ]:
def mask_occ(image_list, error_message, generated_images_count):
    '''
    Masking in the selected occlusion(s) on the given face image
    saves the result in the result-catalog
    Parameters:
        image_list : filename of all face images that are to be processed
    The routine does the occlusion maskin for all face images in the catalog
        (or the one given in the program)
    The masking sequence:
        mask, glass, cap
    '''

    # now, from the dlib we are extracting the method get_frontal_face_detector()
    # and assign that object result to frontal_face_detector to detect face from the image with
    # the help of the 68_face_landmarks.dat model
    frontal_face_detector = dlib.get_frontal_face_detector()

    # Now the dlip shape_predictor class will take model and with the help of that, it will show
    face_landmark_detector = dlib.shape_predictor(model_path)

    if glass:
        glass_mask = cv2.imread(occ_catalogue + '/' +glass, cv2.IMREAD_UNCHANGED)
        if glass_mask.shape[2] == 3: # Add alpha channel if not exist in image
            glass_mask = cv2.cvtColor(glass_mask, cv2.COLOR_BGR2BGRA)
    if mask:
        face_mask = cv2.imread(occ_catalogue+ '/' + mask, cv2.IMREAD_UNCHANGED)
        if face_mask.shape[2] == 3: # Add alpha channel if not exist in image
            face_mask = cv2.cvtColor(face_mask, cv2.COLOR_BGR2BGRA)
    if cap:
        cap_mask = cv2.imread(occ_catalogue + '/' + cap, cv2.IMREAD_UNCHANGED)
        if cap_mask.shape[2] == 3: # Add alpha channel if not exist in image
            cap_mask = cv2.cvtColor(cap_mask, cv2.COLOR_BGR2BGRA)

    ''' For each face image filenames in the list '''
    for image_file_name in tqdm(image_list, file=sys.stdout):
        face_img = cv2.imread(face_image_catalogue + '/' + image_file_name, cv2.IMREAD_UNCHANGED)
        # check the given face images
        if isinstance(face_img, type(None)):
            error_message.append('Image ' + image_file_name + ' is non-type.')
            continue # skip this image, take the next one
        if len(face_img.shape)<3: # Image content is not supported
            error_message.append('Image ' + image_file_name + ' is not supported.')
            continue # skip this image, take the next one

        image_RGB = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
        if face_img.shape[2] == 3: # Add alpha channel if not exist in image
            face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2BGRA)

        all_faces = frontal_face_detector(image_RGB, 0)

        if len(all_faces) == 0:
            error_message.append('Image ' + image_file_name +

                                 ' does not contain a recognizeable face')
            continue  # skip this image, take the next one
        if len(all_faces) > 1:
            error_message.append('Image ' + image_file_name +

                                 ' does contain more than one face')
            continue  # skip this image, take the next one


        # Ready for processing the face image
        for k in range(0, len(all_faces)):
            # dlib rectangle class will detect faces so that landmarks can apply inside of that area
            face_rectangle_dlib = dlib.rectangle(int(all_faces[k].left()),
                                                 int(all_faces[k].top()),
                                                 int(all_faces[k].right()),
                                                 int(all_faces[k].bottom()))

            # Now we are running a loop on every detected face and putting landmark on that with the help of face_landmark_detector
            detected_landmarks = face_landmark_detector(image_RGB, face_rectangle_dlib)

            # Check if the face image can be processed by this program
            landmarks_ok, error_message = check_if_landmarks_within_image(detected_landmarks,

                                                                          face_img,

                                                                          image_file_name,

                                                                          error_message)
            face_from_front = check_if_face_from_front(detected_landmarks)
            if not face_from_front:
                error_message.append('Image ' + image_file_name +

                                     ' is not taken from front. Can not process the image')
            if not landmarks_ok or not face_from_front:
                continue # skip this image, take the next one
            else: # continue with next face image
                ''' Check if the image has to be rotated '''
                angle = rotate_face_image_angle(face_img, detected_landmarks)
                if angle != 0: # rotating all images whit an angle != 0
                    rotated_img = rotate_and_crop(face_img, angle, True)
                    image_RGB = cv2.cvtColor(rotated_img, cv2.COLOR_BGR2RGB)
                    if len(frontal_face_detector(image_RGB, 0)) == 0:
                        rotated_img = rotate_and_crop(face_img, angle, False)
                        image_RGB = cv2.cvtColor(rotated_img, cv2.COLOR_BGR2RGB)
                    face_img = rotated_img.copy() # img

                    # Redetect landmarks in the rotated image
                    all_faces = frontal_face_detector(image_RGB, 0)
                    face_rectangle_dlib = dlib.rectangle(int(all_faces[k].left()),
                                                         int(all_faces[k].top()),
                                                         int(all_faces[k].right()),
                                                         int(all_faces[k].bottom()))
                    detected_landmarks = face_landmark_detector(image_RGB,

                                                                face_rectangle_dlib)

                    # ...end rotated (and cropped) image...

                # masking the face image with the occlusion given in the program sequence:
                if mask:
                    face_img = mask_face_mask(face_img, face_mask,

                                              detected_landmarks)
                if glass:
                    face_img = mask_glasses(face_img, glass_mask,

                                            detected_landmarks)
                if cap:
                    face_img = mask_cap(face_img, cap_mask, detected_landmarks)

                print ("Image file: ",image_file_name,  " occ: Cap:", cap, " mask: ", mask, "Glass: ", glass)
                # prepare saving of result file:
                ext = ''
                if mask:
                    ext = '_mask'
                if glass:
                    if glass == 'glass.png':
                        ext = ext + '_glass'
                    else:
                        ext = ext + '_sunglass'
                if cap:
                    ext = ext + '_cap'

                # prepare for saving the result file
                file_details = os.path.splitext(image_file_name) # Get filename (without extension)
                new_name = file_details[0] + ext
                new_name_path = result_catalogue + '/' + new_name + file_details[1]
                # save face images with occlusion(s)
                cv2.imwrite(new_name_path, face_img)

                # increment number of successful face image occlusion masking
                generated_images_count += 1
    return error_message, generated_images_count

In [ ]:
error_message = []
generated_images_count = 0

# Check if user has provided cataloges and model file, and if they exist
if not (face_image_catalogue or os.path.isdir(face_image_catalogue)):
    error_message.append('Face image catalog not found!')
if not (occ_catalogue or os.path.isdir(occ_catalogue)):
    error_message.append('Occlusion image catalog not found!')
if not (result_catalogue or os.path.isdir(result_catalogue)):
    error_message.append('Result catalog not found!')
if not (model_path or os.path.isfile(model_path)):
    error_message.append('Missing or not found model file!')

# Check if occlusions are selected and exist
if not (glass or mask or cap):
    error_message.append('No occlusions selected!')
if glass and not os.path.isfile(occ_catalogue + '/' + glass):
    error_message.append('Occlusion file: ' + glass +

                         ' does not exist in the occlusion image catalog')
if mask and not os.path.isfile(occ_catalogue + '/' + mask):
    error_message.append('Occlusion file: ' + mask +

                         ' does not exist in the occlusion image catalog')
if cap and not os.path.isfile(occ_catalogue + '/' + cap):
    error_message.append('Occlusion file: ' + cap +

                         ' does not exist in the occlusion image catalog')

# Make a loop of all images in the image catalog
if face_image_catalogue:
    image_list = []
    for file in os.listdir(face_image_catalogue):
        # Check if file is image and not in processed before
        if file.endswith(('.jpg','.png','.jpeg')) and not any(ext in file for ext in synthetic_file_ext):
            image_list.append(file)

    # Check if there are any face images in the selected catalog
    if not image_list:
        error_message.append('Can not find any genuine images (*.png, *.jpg, *.jpeg)'

                             + ' in the face image catalog')


# If no errors have occured until now, continue with the program
if not error_message:
    # Mask all face images with all given occlusions
    error_message, generated_images_count = mask_occ(image_list,

                                                     error_message,

                                                     generated_images_count)
    # Print the result of the program
    print_result(error_message, generated_images_count)
else:
    # Print all the error messages
    print("--- Run failed ---", file=sys.stderr)
    print(*error_message, sep="\n", file=sys.stderr)